In [113]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [114]:
endpoint = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

res = requests.get(endpoint)

soup = BeautifulSoup(res.text)



In [115]:
us_state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

In [116]:
# def get_all():
#     districts = ['01', '02', ...] 
#     for state in us_state_abbreviations:
#         endpoint = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{districts}&spec=N'

#         res = requests.get(endpoint)

#         soup = BeautifulSoup(res.text)

#         soup.find()

In [117]:
# candidate_elements = soup.find_all('h2')

# # Placeholder for our extracted data

# for element in candidate_elements:
#     text = element.get_text(strip=True)
#     print(text)

In [189]:
# Your existing code
endpoint = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
res = requests.get(endpoint)
soup = BeautifulSoup(res.text, 'html.parser')
pattern_candidate = re.compile(r"(.+?) \((R|D|I)\)( •Incumbent•Winner)?(\(([\d.]+)% of vote\))?")
# Your existing code...
pandas_data = []

# Extract candidate data
for element in soup.find_all('h2'):
    text = element.get_text(strip=True)
    match = pattern_candidate.match(text)
    if match:
        name, party, incumbent_winner, _, vote_percentage = match.groups()
        incumbent = incumbent_winner is not None
        winner = incumbent  # Assumes if they are incumbent they are also the winner
        pandas_data.append({
            "Name": name,
            "Party": party,
            "Incumbent": incumbent,
            "Winner": winner,
            "Vote Percentage": vote_percentage,
            "Raised": None,
            "Spent": None
        })

financial_tables = soup.select('table', _class="Members--table")
# Initialize a list to store extracted data
# Initialize a list to store the financial data
financial_data = []

# Loop through all the tables with class 'Members--table'
for table in soup.find_all('table', class_='Members--table'):
    # Initialize a dictionary to store the raised and spent values for this table
    table_data = {}
    
    # Loop through all the tr elements in the table
    for row in table.find_all('tr'):
        # Get the text content of all td elements in the row
        cols = [col.get_text() for col in row.find_all('td')]
        
        # Check if the first column is 'Raised' or 'Spent', and if so, store the data
        if cols[0] == 'Raised:':
            table_data['Raised'] = cols[1]
        elif cols[0] == 'Spent:':
            table_data['Spent'] = cols[1]
    
    # Append the data dictionary to the financial_data list
    financial_data.append(table_data)

# Check if lengths of pandas_data and financial_data match
if len(pandas_data) == len(financial_data):
    # Update pandas_data with financial data
    for p_data, f_data in zip(pandas_data, financial_data):
        p_data.update(f_data)
else:
    print("Mismatch in the number of entries between candidate data and financial data")

In [191]:
df = pd.DataFrame(pandas_data)
df

,Name,Party,Incumbent,Winner,Vote Percentage,Raised,Spent
0,Mark Green,R,True,True,69.9,"$1,194,960","$935,487"
1,Kiran Sreepada,D,False,False,27.3,"$206,644","$207,191"
2,Ronald Brown,I,False,False,2.2,"$1,750",$0
3,Scott Vieira Jr,I,False,False,0.6,$655,"$1,049"
